# CellDART Example Code: mouse brain 
## (10x Visium of anterior mouse brain + scRNA-seq data of mouse brain)

In [1]:
import os

import numpy as np
import tensorflow as tf  # TensorFlow registers PluggableDevices here.
from tqdm.autonotebook import tqdm
import yaml

from CellDART import da_cellfraction
from src.da_utils import data_loading




/tmp/ipykernel_25567/3903643673.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
tf.config.list_physical_devices()


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
PS_SEEDS = (3679, 343, 25, 234, 98098)
MODEL_SEEDS = (2353, 24385, 284, 86322, 98237)
MODEL_DIR = "model_FINAL"
SEED_OVERRIDE = None

CONFIGS_DIR = "configs"
CONFIG_FNAME = "celldart-final-spotless-ht.yml"

# BOOTSTRAP = False
# BOOTSTRAP_ROUNDS = 10
# BOOTSTRAP_ALPHAS = [0.6, 1 / 0.6]

MODEL_NAME = "CellDART_original"


In [5]:
with open(os.path.join(CONFIGS_DIR, MODEL_NAME, CONFIG_FNAME), "r") as f:
    config = yaml.safe_load(f)

lib_params = config["lib_params"]
data_params = config["data_params"]
model_params = config["model_params"]
train_params = config["train_params"]

rewrite_config = False
if not "pretraining" in train_params:
    train_params["pretraining"] = True
    rewrite_config = True
if not "lr" in train_params:
    train_params["lr"] = 0.001
    rewrite_config = True

if rewrite_config:
    with open(os.path.join(CONFIGS_DIR, MODEL_NAME, CONFIG_FNAME), "w") as f:
        yaml.safe_dump(config, f)

tqdm.write(yaml.safe_dump(config))


data_params:
  all_genes: false
  data_dir: ../AGrEDA/data
  dset: mouse_cortex
  n_markers: 80
  n_mix: 10
  n_spots: 100000
  samp_split: true
  sc_id: GSE115746
  scaler_name: minmax
  st_id: spotless_mouse_cortex
  st_split: false
lib_params:
  manual_seed: 3565242559
model_params:
  celldart_kwargs:
    bn_momentum: 0.1
    emb_dim: 32
  model_version: gen_spotless_oracle-1939
train_params:
  alpha: 0.6
  alpha_lr: 5
  batch_size: 8
  initial_train_epochs: 10
  lr: 0.001
  n_iter: 15000
  pretraining: true
  reverse_val: false



## 1. Data load
### load scanpy data - 10x datasets

In [5]:
def train(ps_seed, model_seed):
    print(f"PS seed: {ps_seed}, model seed: {model_seed}")

    model_folder = data_loading.get_model_rel_path(
        MODEL_NAME,
        model_params["model_version"],
        lib_seed_path=str(model_seed),
        **data_params,
    )

    model_folder = os.path.join(MODEL_DIR, model_folder)
    if not os.path.isdir(model_folder):
        os.makedirs(model_folder)
        print(model_folder)
        
    selected_dir = data_loading.get_selected_dir(
        data_loading.get_dset_dir(
            data_params["data_dir"],
            dset=data_params.get("dset", "dlpfc"),
        ),
        **data_params,
    )
    # Load spatial data
    mat_sp_d, mat_sp_meta_d, st_sample_id_l = data_loading.load_spatial(
        selected_dir,
        **data_params,
    )

    # Load sc data
    sc_mix_d, lab_mix_d, sc_sub_dict, sc_sub_dict2 = data_loading.load_sc(
        selected_dir,
        **data_params,
        seed_int=ps_seed,
    )

    target_d = {}
    if "train" in mat_sp_d:
        # keys of dict are splits
        for split in mat_sp_d:
            target_d[split] = np.concatenate(list(mat_sp_d[split].values()))
    else:
        # keys of subdicts are splits
        for split in next(iter(mat_sp_d.values())):
            target_d[split] = np.concatenate((v[split] for v in mat_sp_d.values()))


    advtrain_folder = os.path.join(model_folder, "advtrain")
    pretrain_folder = os.path.join(model_folder, "pretrain")
    if not os.path.isdir(advtrain_folder):
        os.makedirs(advtrain_folder)
    if not os.path.isdir(pretrain_folder):
        os.makedirs(pretrain_folder)

    if data_params.get("samp_split"):
        tqdm.write(f"Adversarial training for slides {mat_sp_d['train'].keys()}: ")
        save_folder = os.path.join(advtrain_folder, "samp_split")
    else:
        tqdm.write(f"Adversarial training for slides {next(iter(mat_sp_d.keys()))}: ")
        save_folder = os.path.join(advtrain_folder, "one_model")

    if not os.path.isdir(save_folder):
        os.makedirs(save_folder)

    embs, embs_noda, clssmodel, clssmodel_noda = da_cellfraction.train(
        sc_mix_d["train"],
        lab_mix_d["train"],
        target_d["train"],
        alpha=train_params.get("alpha", 0.6),
        alpha_lr=train_params.get("alpha_lr", 5),
        emb_dim=model_params["celldart_kwargs"].get("emb_dim", 64),
        batch_size=train_params.get("batch_size", 512),
        n_iterations=train_params.get("n_iter", 3000),
        initial_train=train_params.get("pretraining", True),
        initial_train_epochs=train_params.get("initial_train_epochs", 10),
        batch_size_initial_train=max(train_params.get("batch_size", 512), 512),
        bn_momentum=1-config["model_params"].get("bn_momentum", 0.01),
        seed=model_seed,
    )

    # Save model

    if not os.path.isdir(os.path.join(save_folder, "final_model")):
        os.makedirs(os.path.join(save_folder, "final_model"))
    if not os.path.isdir(os.path.join(pretrain_folder, "final_model")):
        os.makedirs(os.path.join(pretrain_folder, "final_model"))

    clssmodel_noda.save(os.path.join(pretrain_folder, "final_model", "model"))
    clssmodel.save(os.path.join(save_folder, "final_model", "model"))

    embs_noda.save(os.path.join(pretrain_folder, "final_model", "embs"))
    embs.save(os.path.join(save_folder, "final_model", "embs"))

    with open(os.path.join(model_folder, "config.yml"), "w") as f:
        yaml.safe_dump(config, f)


for ps_seed, model_seed in zip(PS_SEEDS, MODEL_SEEDS):
    train(ps_seed, model_seed)

PS seed: 3679, model seed: 2353
Adversarial training for slides dict_keys(['Eng2019_cortex_svz_fov0', 'Eng2019_cortex_svz_fov1', 'Eng2019_cortex_svz_fov4', 'Eng2019_cortex_svz_fov5', 'Eng2019_cortex_svz_fov6']): 
Train on 100000 samples
Epoch 1/10
100000/100000 [==============================] - 11s 111us/sample - loss: 0.5123 - mae: 0.0244
Epoch 2/10
100000/100000 [==============================] - 1s 12us/sample - loss: 0.2113 - mae: 0.0140
Epoch 3/10
100000/100000 [==============================] - 1s 12us/sample - loss: 0.1711 - mae: 0.0124
Epoch 4/10
100000/100000 [==============================] - 1s 13us/sample - loss: 0.1534 - mae: 0.0116
Epoch 5/10
100000/100000 [==============================] - 1s 10us/sample - loss: 0.1426 - mae: 0.0110
Epoch 6/10
100000/100000 [==============================] - 1s 14us/sample - loss: 0.1341 - mae: 0.0106
Epoch 7/10
100000/100000 [==============================] - 1s 11us/sample - loss: 0.1277 - mae: 0.0103
Epoch 8/10
100000/100000 [=======

/home/wma/miniconda3/envs/CellDART/lib/python3.8/site-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


0.12401113071441651
0.12401113071441651
Iteration 99, source loss =  1.097, discriminator acc = 1.000
Iteration 199, source loss =  0.958, discriminator acc = 1.000
Iteration 299, source loss =  0.954, discriminator acc = 1.000
Iteration 399, source loss =  0.728, discriminator acc = 1.000
Iteration 499, source loss =  0.904, discriminator acc = 1.000
Iteration 599, source loss =  0.750, discriminator acc = 1.000
Iteration 699, source loss =  0.705, discriminator acc = 1.000
Iteration 799, source loss =  0.712, discriminator acc = 0.994
Iteration 899, source loss =  0.717, discriminator acc = 0.838
Iteration 999, source loss =  0.641, discriminator acc = 1.000
Iteration 1099, source loss =  0.637, discriminator acc = 0.971
Iteration 1199, source loss =  0.584, discriminator acc = 1.000
Iteration 1299, source loss =  0.550, discriminator acc = 0.970
Iteration 1399, source loss =  0.486, discriminator acc = 0.961
Iteration 1499, source loss =  0.463, discriminator acc = 0.923
Iteration 1

/home/wma/miniconda3/envs/CellDART/lib/python3.8/site-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


0.11530385658025742
0.11530385658025742
Iteration 99, source loss =  1.214, discriminator acc = 1.000
Iteration 199, source loss =  1.215, discriminator acc = 1.000
Iteration 299, source loss =  0.888, discriminator acc = 1.000
Iteration 399, source loss =  0.889, discriminator acc = 0.998
Iteration 499, source loss =  0.806, discriminator acc = 0.965
Iteration 599, source loss =  0.773, discriminator acc = 1.000
Iteration 699, source loss =  0.717, discriminator acc = 1.000
Iteration 799, source loss =  0.748, discriminator acc = 1.000
Iteration 899, source loss =  0.706, discriminator acc = 1.000
Iteration 999, source loss =  0.615, discriminator acc = 0.984
Iteration 1099, source loss =  0.603, discriminator acc = 0.995
Iteration 1199, source loss =  0.578, discriminator acc = 0.999
Iteration 1299, source loss =  0.552, discriminator acc = 0.964
Iteration 1399, source loss =  0.559, discriminator acc = 0.988
Iteration 1499, source loss =  0.552, discriminator acc = 0.996
Iteration 1

...

0.11997314372777938
0.11997314372777938
Iteration 99, source loss =  0.713, discriminator acc = 1.000
Iteration 199, source loss =  0.983, discriminator acc = 1.000
Iteration 299, source loss =  1.229, discriminator acc = 1.000
Iteration 399, source loss =  0.757, discriminator acc = 1.000
Iteration 499, source loss =  0.775, discriminator acc = 1.000
Iteration 599, source loss =  0.871, discriminator acc = 1.000
Iteration 699, source loss =  0.760, discriminator acc = 1.000
Iteration 799, source loss =  0.748, discriminator acc = 1.000
Iteration 899, source loss =  0.865, discriminator acc = 1.000
Iteration 999, source loss =  0.819, discriminator acc = 1.000
Iteration 1099, source loss =  0.840, discriminator acc = 1.000
Iteration 1199, source loss =  0.712, discriminator acc = 1.000
Iteration 1299, source loss =  0.694, discriminator acc = 0.996
Iteration 1399, source loss =  0.692, discriminator acc = 0.999
Iteration 1499, source loss =  0.675, discriminator acc = 0.879
Iteration 1

/home/wma/miniconda3/envs/CellDART/lib/python3.8/site-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


0.1230966380906105
0.1230966380906105
Iteration 99, source loss =  0.889, discriminator acc = 1.000
Iteration 199, source loss =  0.700, discriminator acc = 1.000
Iteration 299, source loss =  0.917, discriminator acc = 1.000
Iteration 399, source loss =  0.822, discriminator acc = 1.000
Iteration 499, source loss =  0.804, discriminator acc = 1.000
Iteration 599, source loss =  0.848, discriminator acc = 1.000
Iteration 699, source loss =  0.779, discriminator acc = 1.000
Iteration 799, source loss =  0.759, discriminator acc = 0.995
Iteration 899, source loss =  0.822, discriminator acc = 1.000
Iteration 999, source loss =  0.814, discriminator acc = 1.000
Iteration 1099, source loss =  0.764, discriminator acc = 0.989
Iteration 1199, source loss =  0.806, discriminator acc = 0.999
Iteration 1299, source loss =  0.832, discriminator acc = 1.000
Iteration 1399, source loss =  0.809, discriminator acc = 0.996
Iteration 1499, source loss =  0.795, discriminator acc = 0.999
Iteration 159

/home/wma/miniconda3/envs/CellDART/lib/python3.8/site-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


0.11728710390806198
0.11728710390806198
Iteration 99, source loss =  1.103, discriminator acc = 0.999
Iteration 199, source loss =  0.809, discriminator acc = 1.000
Iteration 299, source loss =  0.880, discriminator acc = 1.000
Iteration 399, source loss =  0.867, discriminator acc = 1.000
Iteration 499, source loss =  0.820, discriminator acc = 0.999
Iteration 599, source loss =  0.781, discriminator acc = 0.910
Iteration 699, source loss =  0.770, discriminator acc = 0.996
Iteration 799, source loss =  0.815, discriminator acc = 0.769
Iteration 899, source loss =  0.714, discriminator acc = 0.991
Iteration 999, source loss =  0.743, discriminator acc = 1.000
Iteration 1099, source loss =  0.721, discriminator acc = 1.000
Iteration 1199, source loss =  0.728, discriminator acc = 0.993
Iteration 1299, source loss =  0.732, discriminator acc = 1.000
Iteration 1399, source loss =  0.752, discriminator acc = 0.996
Iteration 1499, source loss =  0.798, discriminator acc = 0.999
Iteration 1

## Eval


In [10]:
import argparse
import datetime
import logging

from src.da_models.model_utils.utils import get_metric_ctp
from src.da_utils.evaluator import Evaluator


metric_ctp = get_metric_ctp("cos")

def _device_to_str(device):
    return device.name.lstrip("/physical_device:")

def get_device(cuda_index=None):
    if cuda_index is None:
        return _device_to_str(tf.config.list_physical_devices("GPU")[0])

    cuda_index = int(cuda_index)

    if cuda_index < 0:
        return _device_to_str(tf.config.list_physical_devices("CPU")[0])

    devices=tf.config.list_physical_devices("GPU")

    if len(devices) > cuda_index:
        return _device_to_str(devices[cuda_index])
    if len(devices) > 0:
        warnings.warn("GPU ordinal not valid; using default", category=UserWarning, stacklevel=2)
        return _device_to_str(devices[0])

    warnings.warn("Using CPU", category=UserWarning, stacklevel=2)
    return _device_to_str(tf.config.list_physical_devices("CPU")[0])

def main(args):
    evaluator = Evaluator(vars(args), metric_ctp)
    evaluator.eval_spots()
    evaluator.evaluate_embeddings()
    evaluator.eval_sc()

    evaluator.produce_results()

for ps_seed, model_seed in zip(PS_SEEDS, MODEL_SEEDS):
    parser = argparse.ArgumentParser(description="Evaluates.")
    parser.add_argument("--pretraining", "-p", action="store_true", help="force pretraining")
    parser.add_argument("--modelname", "-n", type=str, default="ADDA", help="model name")
    parser.add_argument("--milisi", "-m", action="store_false", help="no milisi")
    parser.add_argument("--config_fname", "-f", type=str, help="Name of the config file to use")
    parser.add_argument("--configs_dir", "-cdir", type=str, default="configs", help="config dir")
    parser.add_argument(
        "--njobs", type=int, default=1, help="Number of jobs to use for parallel processing."
    )
    parser.add_argument("--cuda", "-c", default=None, help="GPU index to use")
    parser.add_argument("--tmpdir", "-d", default=None, help="optional temporary results directory")
    parser.add_argument("--test", "-t", action="store_true", help="test mode")
    parser.add_argument(
        "--early_stopping",
        "-e",
        action="store_true",
        help="evaluate early stopping. Default: False",
    )
    parser.add_argument(
        "--reverse_val",
        "-r",
        action="store_true",
        help="use best model through reverse validation. Will use provided"
        "config file to search across models, then use the one loaded. Default: False",
    )
    parser.add_argument("--model_dir", default="model", help="model directory")
    parser.add_argument("--results_dir", default="results", help="results directory")
    parser.add_argument(
        "--seed_override",
        default=None,
        help="seed to use for torch and numpy; overrides that in config file",
    )
    parser.add_argument(
        "--ps_seed",
        default=-1,
        help="specific pseudospot seed to use; default of -1 corresponds to 623",
    )

    args = parser.parse_args([
        f"--modelname={MODEL_NAME}",
        f"--config_fname={CONFIG_FNAME}",
        "--njobs=-1",
        "--test",
        f"--model_dir={MODEL_DIR}",
        "--results_dir=results_FINAL",
        f"--seed_override={model_seed}",
        f"--ps_seed={ps_seed}",
        f"--cuda=-1"
    ])

    script_start_time = datetime.datetime.now(datetime.timezone.utc)
    logger = logging.getLogger(__name__)
    logging.basicConfig(
        level=logging.WARNING,
        format="%(asctime)s:%(levelname)s:%(name)s:%(message)s",
    )
    with tf.device(get_device(args.cuda)):
        a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
        b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
        c = tf.matmul(a, b)

        print(c)

        main(args)
    print("Script run time:", datetime.datetime.now(datetime.timezone.utc) - script_start_time)

-1
Tensor("MatMul_1:0", shape=(2, 2), dtype=float32, device=/device:CPU:0)
Evaluating CellDART_original on with 16 jobs
Using library config:
None
Loading config celldart-final-spotless-ht.yml ... 
data_params:
  all_genes: false
  data_dir: ../AGrEDA/data
  dset: mouse_cortex
  n_markers: 80
  n_mix: 10
  n_spots: 100000
  samp_split: true
  sc_id: GSE115746
  scaler_name: minmax
  st_id: spotless_mouse_cortex
  st_split: false
lib_params:
  manual_seed: 3565242559
model_params:
  celldart_kwargs:
    bn_momentum: 0.1
    emb_dim: 32
  model_version: gen_spotless_oracle-1939
train_params:
  alpha: 0.6
  alpha_lr: 5
  batch_size: 8
  initial_train_epochs: 10
  lr: 0.001
  n_iter: 15000
  pretraining: true
  reverse_val: false

Saving results to results_FINAL/CellDART_original/mouse_cortex/GSE115746_spotless_mouse_cortex/80markers/10mix_100000spots/minmax/gen_spotless_oracle-1939/2353 ...
Loading Data


/home/wma/CellDART/src/da_utils/evaluator.py:159: UserWarning: Output folder exists. Will overwrite.
  self.results_folder = self.temp_folder_holder.set_output_folder(


Loading ST adata: 
Getting predictions: 


/home/wma/miniconda3/envs/CellDART/lib/python3.8/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Calculating domain shift for Eng2019_cortex_svz_fov0: TRAIN |

...

/home/wma/miniconda3/envs/CellDART/lib/python3.8/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


 milisi rf50 | VAL | milisi rf50 | TEST | milisi rf50 | 
Calculating domain shift for Eng2019_cortex_svz_fov1: TRAIN | milisi rf50 | VAL | milisi rf50 | TEST | milisi rf50 | 
Calculating domain shift for Eng2019_cortex_svz_fov4: TRAIN | milisi rf50 | VAL | milisi rf50 | TEST | milisi rf50 | 
Calculating domain shift for Eng2019_cortex_svz_fov5: TRAIN | milisi rf50 | VAL | milisi rf50 | TEST | milisi rf50 | 
Calculating domain shift for Eng2019_cortex_svz_fov6: TRAIN | milisi rf50 | VAL | milisi rf50 | TEST | milisi rf50 | 
Calculating domain shift for Eng2019_cortex_svz_fov2: TRAIN | milisi rf50 | VAL | milisi rf50 | TEST | milisi rf50 | 
Calculating domain shift for Eng2019_cortex_svz_fov3: TRAIN | milisi rf50 | VAL | milisi rf50 | TEST | milisi rf50 | 


/home/wma/miniconda3/envs/CellDART/lib/python3.8/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
/home/wma/miniconda3/envs/CellDART/lib/python3.8/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


                                                        Pseudospots (Cosine Distance)  \
                                                                                train   
                       SC Split Sample ID                                               
Before DA              train    Eng2019_cortex_svz_fov0                      0.035810   
                                Eng2019_cortex_svz_fov1                      0.035810   
                                Eng2019_cortex_svz_fov4                      0.035810   
                                Eng2019_cortex_svz_fov5                      0.035810   
                                Eng2019_cortex_svz_fov6                      0.035810   
                       val      Eng2019_cortex_svz_fov2                      0.035810   
                       test     Eng2019_cortex_svz_fov3                      0.035810   
After DA (final model) train    Eng2019_cortex_svz_fov0                      0.078453   
                     

/home/wma/CellDART/src/da_utils/output_utils.py:119: UserWarning: Destination exists. Will overwrite.
  warnings.warn("Destination exists. Will overwrite.")
